In [17]:
import os
import psycopg2
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.preprocessing import StandardScaler
import joblib

# Configurações gerais
BATCH_SIZE = 10000
OUTPUT_DIR_VITALS = 'output/vitals_batches'
OUTPUT_DIR_LABS = 'output/labs_batches'

os.makedirs(OUTPUT_DIR_VITALS, exist_ok=True)
os.makedirs(OUTPUT_DIR_LABS, exist_ok=True)

conn = psycopg2.connect(
    dbname="mimiciv",
    user="uti_user",
    password="s0f4C1#4",
    host="localhost",
    port="5432"
)
print("✅ Conexão ao banco estabelecida.")


✅ Conexão ao banco estabelecida.


DETAIL:  The database was created using collation version 2.35, but the operating system provides version 2.39.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE mimiciv REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


In [18]:
cursor = conn.cursor()
cursor.execute("ROLLBACK")
cursor.execute("""
CREATE TEMP TABLE todas_utis AS
SELECT 
    i.subject_id,
    i.hadm_id,
    i.stay_id,
    i.intime,
    i.outtime,
    EXTRACT(EPOCH FROM (i.outtime - i.intime)) / 60 AS duracao_minutos
FROM mimiciv_icu.icustays i
WHERE EXTRACT(EPOCH FROM (i.outtime - i.intime)) > 3600;
""")
conn.commit()

df_total = pd.read_sql("SELECT COUNT(*) AS total_admissoes FROM todas_utis;", conn)
print(f"✅ Total admissões na UTI (>1h): {df_total['total_admissoes'][0]}")

todas_utis = pd.read_sql("SELECT DISTINCT stay_id FROM todas_utis", conn)['stay_id'].tolist()


✅ Total admissões na UTI (>1h): 73073


/tmp/ipykernel_102500/1640830532.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_total = pd.read_sql("SELECT COUNT(*) AS total_admissoes FROM todas_utis;", conn)
/tmp/ipykernel_102500/1640830532.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  todas_utis = pd.read_sql("SELECT DISTINCT stay_id FROM todas_utis", conn)['stay_id'].tolist()


In [19]:
vars_alta_freq = [
    'heart_rate', 'sbp', 'dbp', 'mbp', 'resp_rate', 'spo2',
    'lab_53085', 'lab_51580', 'lab_52642', 'lab_51002', 'lab_52116', 'lab_51623',
    'lab_50928', 'lab_52117', 'lab_50855', 'lab_52546', 'lab_53161', 'lab_53180',
    'lab_52142', 'lab_51266', 'lab_52144', 'lab_51631', 'lab_51638', 'lab_51640',
    'lab_51647', 'lab_51643', 'lab_50975', 'lab_51292', 'lab_51290', 'lab_51291',
    'lab_52551', 'lab_51568', 'lab_51569', 'lab_51570', 'lab_51464', 'lab_51966'
]

vars_media_freq = [
    'temperature', 'glucose', 'lab_50908', 'lab_50915', 'lab_50856', 'lab_50803',
    'lab_50805', 'lab_50808', 'lab_50809', 'lab_50813'
]

vars_baixa_freq = [
    'lab_50861', 'lab_50862', 'lab_50883', 'lab_50884', 'lab_50885', 'lab_50910',
    'lab_50924', 'lab_50963', 'lab_51003', 'lab_50889', 'lab_51214', 'lab_50878',
    'lab_50912', 'lab_51265', 'lab_50931', 'lab_50935', 'lab_51222', 'lab_51223',
    'lab_50852', 'lab_50971', 'lab_50983', 'lab_50990', 'lab_50967', 'lab_50968',
    'lab_50969', 'lab_50960', 'lab_50966', 'lab_50970', 'lab_51099', 'lab_51006',
    'lab_51274', 'lab_51275', 'lab_51196'
]

vasopressor_vars = [
    'dopamine', 'epinephrine', 'norepinephrine', 'phenylephrine',
    'vasopressin', 'dobutamine', 'milrinone'
]

print(f"Alta frequência: {len(vars_alta_freq)} variáveis")
print(f"Média frequência: {len(vars_media_freq)} variáveis")
print(f"Baixa frequência: {len(vars_baixa_freq)} variáveis")
print(f"Farmacológicas: {len(vasopressor_vars)} variáveis")

Alta frequência: 36 variáveis
Média frequência: 10 variáveis
Baixa frequência: 33 variáveis
Farmacológicas: 7 variáveis


In [20]:
# Valores normais (default values) para variáveis (conforme artigo/excel)
NORMAL_VALUES = {
    'heart_rate': 70,
    'sbp': 125,
    'dbp': 75,
    'mbp': 90,
    'resp_rate': 12,
    'temperature': 37,
    'spo2': 98,
    'glucose': 5,  # mmol/L, conforme tabela (se precisar converter, ajuste)

}

for var in vasopressor_vars:
    NORMAL_VALUES[var] = 0

# Parâmetros mediana e IQR obtidos do cálculo com amostra (ou valores do artigo)
PARAMETROS_IMPUTACAO = {
    'lab_50861': {'default': 1, 'range': (0, 15)},               # Lactate, Blood
    'lab_50862': {'default': 1, 'range': (0, 15)},               # Lactate, Arterial
    'lab_53085': {'default': 1, 'range': (0, 15)},               # Lactate, Venous
    'lab_50908': {'default': 1, 'range': (0, 8)},                # INR (PT)
    'lab_51580': {'default': 1, 'range': (0, 8)},                # INR (PT), Arterial
    'lab_50963': {'default': 4, 'range': (0, 600)},              # C-Reactive Protein
    'lab_50889': {'default': 4, 'range': (2, 12)},               # Potassium, Serum or Plasma
    'lab_52116': {'default': 4, 'range': (2, 12)},               # Potassium, Blood
    'lab_51623': {'default': 140, 'range': (105, 170)},          # Sodium, Serum or Plasma
    'lab_50928': {'default': 140, 'range': (105, 170)},          # Sodium, Blood
    'lab_52117': {'default': 140, 'range': (105, 170)},          # Sodium, Blood
    'lab_51214': {'default': 24, 'range': (0, 50)},              # Bicarbonate, Serum
    'lab_50878': {'default': 103, 'range': (60, 140)},           # Chloride, Serum or Plasma
    'lab_50855': {'default': 1, 'range': (0, 3)},                # Calcium, Serum
    'lab_50912': {'default': 1, 'range': (0, 5)},                # Magnesium, Serum or Plasma
    'lab_52546': {'default': 1, 'range': (0, 5)},                # Phosphate, Serum
    'lab_53161': {'default': 1, 'range': (0, 5)},                # Phosphate, Blood
    'lab_53180': {'default': 1, 'range': (0, 5)},                # Phosphate, Urine
    'lab_52142': {'default': 2, 'range': (0, 1000)},             # Bilirubin, Total Serum
    'lab_51265': {'default': 2, 'range': (0, 800)},              # Bilirubin, Direct Serum
    'lab_52144': {'default': 2, 'range': (0, 1000)},             # Bilirubin, Blood (assumido igual Total)
    'lab_50931': {'default': 38, 'range': (0, 70)},              # Albumin, Serum or Plasma
    'lab_51631': {'default': 85, 'range': (0, 3000)},            # Alkaline Phosphatase, Serum
    'lab_51638': {'default': 25, 'range': (0, 12000)},           # Alanine Aminotransferase (ALT), Serum
    'lab_51640': {'default': 25, 'range': (0, 30000)},           # Aspartate Aminotransferase (AST), Serum
    'lab_51223': {'default': 50, 'range': (0, 30000)},           # Creatine Kinase, Serum
    'lab_50856': {'default': 5, 'range': (0, 60)},               # Urea Nitrogen (BUN), Serum
    'lab_51647': {'default': 200, 'range': (0, 15000)},          # Troponin T, Serum (valor estimado)
    'lab_50852': {'default': 200, 'range': (0, 15000)},          # Troponin I, Serum (valor estimado)
    'lab_50983': {'default': 135, 'range': (20, 200)},           # Hemoglobin, Blood
    'lab_50968': {'default': 300, 'range': (0, 1500)},           # Platelet Count, Blood (estimado)
    'lab_50960': {'default': 99, 'range': (60, 130)},            # Mean Corpuscular Volume (MCV), Blood
    'lab_51196': {'default': 7, 'range': (6.5, 7.8)},             # Blood Gas, pH
    'lab_52551': {'default': 40, 'range': (3, 100)},              # Blood Gas, pCO2
    'lab_51568': {'default': 87, 'range': (20, 500)},             # Blood Gas, pO2
    'lab_51569': {'default': 96, 'range': (10, 100)},             # Blood Gas, Oxygen Saturation
    'lab_51570': {'default': 24, 'range': (0, 50)},               # Blood Gas, Bicarbonate
    'lab_51966': {'default': 1, 'range': (0, 15)},                # Blood Gas, Lactate
    'lab_50803': {'default': 140, 'range': (105, 170)},           # Blood Gas, Sodium
    'lab_50805': {'default': 4, 'range': (2, 12)},                # Blood Gas, Potassium
    'lab_50808': {'default': 1, 'range': (0, 3)},                 # Blood Gas, Calcium
    'lab_50809': {'default': 135, 'range': (20, 200)},            # Blood Gas, Hemoglobin
    'lab_50813': {'default': 45, 'range': (30, 60)},              # Blood Gas, Hematocrit (aproximado)
}


print("✅ Valores normais e parâmetros de imputação configurados.")

✅ Valores normais e parâmetros de imputação configurados.


In [21]:
itemids_labs = [
    50861, 50862, 53085, 50908, 51580, 50883, 50884, 50885, 50910, 50924,
    50963, 50915, 52642, 51002, 51003, 50889, 52116, 51623, 50928, 52117,
    51214, 50878, 50855, 50912, 52546, 53161, 53180, 52142, 51265, 51266,
    52144, 50931, 50935, 51631, 51638, 51640, 51222, 51223, 50856, 51647,
    50852, 51643, 50971, 50983, 50990, 50967, 50968, 50969, 50960, 50966,
    50970, 50975, 51099, 51006, 51274, 51275, 51292, 51290, 51291, 50963,
    51196, 52551, 50915, 51568, 51569, 51570, 51464, 51966, 50803, 50805,
    50808, 50809, 50813
]

def value_empty(size, default_val, dtype=None):
    """ Retorna vetor preenchido com valor default """
    if dtype is not None:
        tmp_arr = np.empty(size, dtype=dtype)
    else:
        tmp_arr = np.empty(size)
    tmp_arr[:] = default_val
    return tmp_arr

def empty_nan(size):
    """ Retorna vetor preenchido com NaN """
    arr = np.empty(size)
    arr[:] = np.nan
    return arr

def impute_forward_fill_simple(observ_ts, observ_val, time_grid, global_impute_val=np.nan):
    """
    Forward-fill simples para preenchimento no grid uniforme de tempo.
    Se não houver dado anterior, usa global_impute_val (NaN para sinais vitais normais, 0 para farmacológicas).
    """
    n = len(time_grid)
    pred_values = np.full(n, global_impute_val, dtype=np.float32)
    last_val = global_impute_val
    i_obs = 0

    for i_pred, t_pred in enumerate(time_grid):
        # Avança no vetor de observações enquanto timestamps <= timestamp da previsão
        while i_obs < len(observ_ts) and observ_ts[i_obs] <= t_pred:
            last_val = observ_val[i_obs]
            i_obs += 1

        pred_values[i_pred] = last_val

    return pred_values

def imputar_batch_simples(df_batch, variaveis, global_impute_vals):
    df_imputado_list = []

    for stay_id, grupo in df_batch.groupby('stay_id'):
        grupo = grupo.sort_values('charttime').reset_index(drop=True)
        ts_raw = grupo['charttime'].values.astype('datetime64[s]').astype(np.int64)
        t0, tmax = ts_raw[0], ts_raw[-1]
        timegrid = np.arange(t0, tmax + 300, 300)  # passo de 5 minutos

        dict_imputado = {
            'stay_id': [stay_id] * len(timegrid),
            'charttime': pd.to_datetime(timegrid, unit='s')
        }

        for var in variaveis:
            if var not in grupo.columns or grupo[var].dropna().empty:
                # Nenhum dado observado: preenche com valor global_impute_val
                fill_val = global_impute_vals.get(var, np.nan)
                dict_imputado[var] = value_empty(len(timegrid), fill_val)
                dict_imputado[f'{var}_imputed'] = np.ones(len(timegrid), dtype=int)  # 1 indica imputado
                continue

            raw_vals = grupo[var].values
            pred_vals = impute_forward_fill_simple(ts_raw, raw_vals, timegrid, global_impute_vals.get(var, np.nan))
            dict_imputado[var] = pred_vals

            # Máscara: 0 para imputado, 1 para observado
            mask = np.zeros(len(timegrid), dtype=int)
            idxs = np.searchsorted(timegrid, ts_raw)
            idxs = idxs[idxs < len(mask)]
            mask[idxs] = 1
            dict_imputado[f'{var}_imputed'] = 1 - mask  # 1 se imputado, 0 se observado

        df_imputado_list.append(pd.DataFrame(dict_imputado))

    return pd.concat(df_imputado_list, ignore_index=True)

# Exemplo de uso para um batch (conectado a banco e dataframe já definidos)
def processar_batch_vitals(batch_stay_ids):
    batch_ids_str = ",".join(map(str, batch_stay_ids))
    query_vitals = f"""
    SELECT
        vs.stay_id,
        vs.charttime,
        vs.heart_rate,
        vs.sbp,
        vs.dbp,
        vs.mbp,
        vs.resp_rate,
        vs.temperature,
        vs.spo2,
        vs.glucose,
        vs.dopamine,
        vs.epinephrine,
        vs.norepinephrine,
        vs.phenylephrine,
        vs.vasopressin,
        vs.dobutamine,
        vs.milrinone
    FROM mimiciv_derived.vitalsign vs
    WHERE vs.stay_id IN ({batch_ids_str})
    ORDER BY vs.stay_id, vs.charttime;
    """
    print(f"Consultando sinais vitais batch com {len(batch_stay_ids)} stays...")
    df_batch = pd.read_sql(query_vitals, conn)
    df_batch['charttime'] = pd.to_datetime(df_batch['charttime'])
    print(f"Consulta concluída. Linhas: {len(df_batch)}")

    variaveis = list(NORMAL_VALUES.keys())
    df_imputado = imputar_batch_simples(df_batch, variaveis, NORMAL_VALUES)
    print(f"Imputação forward-fill simples concluída para batch com {len(batch_stay_ids)} stays. Linhas após imputação: {len(df_imputado)}")

    return df_imputado


In [ ]:
def processar_batch_labs(batch_stay_ids):
    batch_ids_str = ",".join(map(str, batch_stay_ids))
    query_labs = f"""
    SELECT
        icu.stay_id,
        le.charttime,
        le.itemid,
        le.valuenum
    FROM mimiciv_hosp.labevents le
    JOIN mimiciv_icu.icustays icu ON le.subject_id = icu.subject_id AND le.hadm_id = icu.hadm_id1
    WHERE le.itemid IN ({','.join(map(str, itemids_labs))})
      AND icu.stay_id IN (SELECT stay_id FROM todas_utis)
      AND le.valuenum IS NOT NULL
      AND icu.stay_id IN ({batch_ids_str})
    ORDER BY icu.stay_id, le.charttime;
    """
    print(f"Consultando exames laboratoriais batch com {len(batch_stay_ids)} stays...")
    df_labs = pd.read_sql(query_labs, conn)
    df_labs['charttime'] = pd.to_datetime(df_labs['charttime'])
    print(f"Consulta concluída. Linhas: {len(df_labs)}")

    # Pivotar para formato wide: uma linha por (stay_id, charttime) com colunas lab_<itemid>
    df_pivot = df_labs.pivot_table(index=['stay_id', 'charttime'], columns='itemid', values='valuenum')
    df_pivot.columns = [f'lab_{col}' for col in df_pivot.columns]
    df_pivot = df_pivot.reset_index()
    df_pivot = df_pivot.sort_values(['stay_id', 'charttime']).reset_index(drop=True)

    # Variáveis laboratorias para imputar
    variaveis = [col for col in df_pivot.columns if col.startswith('lab_')]

    # Imputação forward-fill simples conforme artigo
    df_imputado = imputar_batch_simples(df_pivot, variaveis, NORMAL_VALUES)

    print(f"Imputação forward-fill simples concluída para batch labs com {len(batch_stay_ids)} stays. Linhas após imputação: {len(df_imputado)}")

    return df_imputado




In [23]:
def is_batch_processed(batch_idx, output_dir):
    done_flag = os.path.join(output_dir, f'batch_{batch_idx:04d}.parquet.done')
    return os.path.exists(done_flag)

def save_batch(df, batch_idx, output_dir):
    filename = os.path.join(output_dir, f'batch_{batch_idx:04d}.parquet')
    df.to_parquet(filename)
    with open(filename + '.done', 'w') as f:
        f.write('done')
    print(f"Batch {batch_idx} salvo e marcado como concluído.")


In [24]:
batches = [todas_utis[i:i+BATCH_SIZE] for i in range(0, len(todas_utis), BATCH_SIZE)]

# Processar batches vitals
for batch_idx, batch_stays in enumerate(batches):
    if is_batch_processed(batch_idx, OUTPUT_DIR_VITALS):
        print(f"Batch vitals {batch_idx} já processado, pulando...")
        continue
    df_imputado = processar_batch_vitals(batch_stays)
    save_batch(df_imputado, batch_idx, OUTPUT_DIR_VITALS)

# Processar batches labs
for batch_idx, batch_stays in enumerate(batches):
    if is_batch_processed(batch_idx, OUTPUT_DIR_LABS):
        print(f"Batch labs {batch_idx} já processado, pulando...")
        continue
    df_imputado_labs = processar_batch_labs(batch_stays)
    save_batch(df_imputado_labs, batch_idx, OUTPUT_DIR_LABS)


/tmp/ipykernel_102500/619788631.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_labs = pd.read_sql(query_labs, conn)


Batch vitals 0 já processado, pulando...
Batch vitals 1 já processado, pulando...
Batch vitals 2 já processado, pulando...
Batch vitals 3 já processado, pulando...
Batch vitals 4 já processado, pulando...
Batch vitals 5 já processado, pulando...
Batch vitals 6 já processado, pulando...
Batch vitals 7 já processado, pulando...
Batch labs 0 já processado, pulando...
Batch labs 1 já processado, pulando...
Consultando exames laboratoriais batch com 10000 stays...


Consulta concluída. Linhas: 1707642


/tmp/ipykernel_102500/2559897179.py:82: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(df_imputado_list, ignore_index=True)


Imputação forward-fill simples concluída para batch labs com 10000 stays. Linhas após imputação: 28414322
Batch 2 salvo e marcado como concluído.
Consultando exames laboratoriais batch com 10000 stays...


...

Imputação forward-fill simples concluída para batch labs com 10000 stays. Linhas após imputação: 28757807
Batch 6 salvo e marcado como concluído.
Consultando exames laboratoriais batch com 3073 stays...


/tmp/ipykernel_102500/619788631.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_labs = pd.read_sql(query_labs, conn)


Consulta concluída. Linhas: 543186


/tmp/ipykernel_102500/2559897179.py:82: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(df_imputado_list, ignore_index=True)


Imputação forward-fill simples concluída para batch labs com 3073 stays. Linhas após imputação: 8887413
Batch 7 salvo e marcado como concluído.


In [10]:
def processar_vasopressores_e_marcar_falencia(df_merge):
    print(f"✅ Iniciando processamento dos vasopressores e marcação de falência.")

    query_vasoact = """
    SELECT 
        stay_id,
        starttime,
        endtime,
        dopamine,
        epinephrine,
        norepinephrine,
        phenylephrine,
        vasopressin,
        dobutamine,
        milrinone
    FROM mimiciv_derived.vasoactive_agent
    WHERE stay_id IN (SELECT DISTINCT stay_id FROM todas_utis);
    """

    print(f"Consultando dados de vasopressores...")

    df_vasoact = pd.read_sql(query_vasoact, conn)
    df_vasoact['starttime'] = pd.to_datetime(df_vasoact['starttime'])
    df_vasoact['endtime'] = pd.to_datetime(df_vasoact['endtime'])

    vaso_cols = ['dopamine', 'epinephrine', 'norepinephrine', 'phenylephrine',
                 'vasopressin', 'dobutamine', 'milrinone']

    print(f"✅ Dados de vasopressores carregados. Total de registros: {len(df_vasoact)}")

    df_merge['stay_id'] = df_merge['stay_id'].astype(int)
    df_merge['charttime'] = pd.to_datetime(df_merge['charttime'])
    df_vasoact['stay_id'] = df_vasoact['stay_id'].astype(int)

    df_merge = df_merge.dropna(subset=['charttime'])
    df_vasoact = df_vasoact.dropna(subset=['starttime'])

    print(f"✅ Dados preparados: {len(df_merge)} stays vitais e {len(df_vasoact)} stays de vasopressores.")

    df_merge = df_merge.sort_values(['stay_id', 'charttime']).reset_index(drop=True)
    df_vasoact = df_vasoact.sort_values(['stay_id', 'starttime']).reset_index(drop=True)

    dfs_merged = []

    print(f"✅ Iniciando merge dos dados de vasopressores com sinais vitais...")

    for sid in df_merge['stay_id'].unique():
        print(f"Processando stay_id {sid}...")

        df_sid = df_merge[df_merge['stay_id'] == sid]
        df_vaso_sid = df_vasoact[df_vasoact['stay_id'] == sid]

        if df_vaso_sid.empty:
            print(f"⚠️ Sem dados de vasopressores para stay_id {sid}, preenchendo com NA.")
            df_sid.loc[:, ['starttime', 'endtime'] + vaso_cols] = pd.NA
            dfs_merged.append(df_sid)
            continue

        merged_sid = pd.merge_asof(
            df_sid, df_vaso_sid[['stay_id', 'starttime', 'endtime'] + vaso_cols],
            left_on='charttime', right_on='starttime',
            by='stay_id', direction='backward', tolerance=pd.Timedelta('2D')
        )
        dfs_merged.append(merged_sid)

    df_vaso_merged = pd.concat(dfs_merged, ignore_index=True)

    print(f"✅ Merge concluído. Total de registros após merge: {len(df_vaso_merged)}")

    # Condições para marcação da falência
    cond_tempo = (df_vaso_merged['charttime'] >= df_vaso_merged['starttime']) & \
                 (df_vaso_merged['charttime'] <= df_vaso_merged['endtime'])
    cond_vaso = df_vaso_merged[vaso_cols].notna().any(axis=1)
    cond_mbp = df_vaso_merged['mbp'] < 65
    cond_lactato = df_vaso_merged.get('lab_50813', pd.Series(0)) >= 2  # lactato lab_50813

    print(f"✅ Aplicando condições para marcar falência...")

    df_vaso_merged['falencia'] = 0
    idx_falencia = df_vaso_merged.index[cond_tempo & cond_vaso & (cond_mbp | cond_lactato)]
    df_vaso_merged.loc[idx_falencia, 'falencia'] = 1

    print(f"✅ Falência circulatória marcada. Total casos: {df_vaso_merged['falencia'].sum()}")

    return df_vaso_merged


In [11]:
import os
import psycopg2
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.preprocessing import StandardScaler
import joblib
import gc

conn = psycopg2.connect(
    dbname="mimiciv",
    user="uti_user",
    password="s0f4C1#4",
    host="localhost",
    port="5432"
)
print("✅ Conexão ao banco estabelecida.")

cursor = conn.cursor()
cursor.execute("ROLLBACK")  # para garantir estado limpo
cursor.execute("""
CREATE TEMP TABLE todas_utis AS
SELECT 
    i.subject_id,
    i.hadm_id,
    i.stay_id,
    i.intime,
    i.outtime,
    EXTRACT(EPOCH FROM (i.outtime - i.intime)) / 60 AS duracao_minutos
FROM mimiciv_icu.icustays i
WHERE EXTRACT(EPOCH FROM (i.outtime - i.intime)) > 3600;
""")
conn.commit()
cursor.close()

df_falencia = processar_vasopressores_e_marcar_falencia(df_merge)


DETAIL:  The database was created using collation version 2.35, but the operating system provides version 2.39.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE mimiciv REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.
/tmp/ipykernel_103629/3746042496.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_vasoact = pd.read_sql(query_vasoact, conn)


✅ Conexão ao banco estabelecida.
✅ Iniciando processamento dos vasopressores e marcação de falência.
Consultando dados de vasopressores...
✅ Dados de vasopressores carregados. Total de registros: 665528
✅ Dados preparados: 35225737 stays vitais e 665528 stays de vasopressores.


: 

In [ ]:
import os
import psycopg2
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.preprocessing import StandardScaler
import joblib
import gc

conn = psycopg2.connect(
    dbname="mimiciv",
    user="uti_user",
    password="s0f4C1#4",
    host="localhost",
    port="5432"
)
print("✅ Conexão ao banco estabelecida.")

# Ajuste os caminhos conforme necessário
OUTPUT_DIR_VITALS = 'output/vitals_batches'
OUTPUT_DIR_LABS = 'output/labs_batches'
OUTPUT_DIR_FALENCIA = 'output/falencia_batches'

os.makedirs(OUTPUT_DIR_FALENCIA, exist_ok=True)

for batch_idx in range(8):  # 0 a 7
    print(f"\n🔁 Processando batch {batch_idx}...")

    vitals_path = os.path.join(OUTPUT_DIR_VITALS, f'batch_{batch_idx:04d}.parquet')
    labs_path = os.path.join(OUTPUT_DIR_LABS, f'batch_{batch_idx:04d}.parquet')
    falencia_path = os.path.join(OUTPUT_DIR_FALENCIA, f'batch_{batch_idx:04d}.parquet')

    if not os.path.exists(vitals_path) or not os.path.exists(labs_path):
        print(f"❌ Arquivo ausente no batch {batch_idx}, pulando.")
        continue

    # Carregar os arquivos
    df_v = pd.read_parquet(vitals_path)
    df_l = pd.read_parquet(labs_path)

    # Merge vitals + labs
    df_merge = pd.merge(df_v, df_l, on=['stay_id', 'charttime'], how='outer')
    df_merge = df_merge.sort_values(['stay_id', 'charttime']).reset_index(drop=True)


    # Salvar resultado
    df_falencia.to_parquet(falencia_path)
    print(f"✅ Batch {batch_idx} salvo com falência. Linhas: {len(df_falencia)}")

    # Liberação de memória
    del df_v, df_l, df_merge, df_falencia
    gc.collect()


DETAIL:  The database was created using collation version 2.35, but the operating system provides version 2.39.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE mimiciv REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


✅ Conexão ao banco estabelecida.

🔁 Processando batch 0...


In [26]:
def criar_features_instabilidade(df, event_col='falencia'):
    df = df.sort_values(['stay_id', 'charttime']).reset_index(drop=True)
    df['estado_atual'] = df[event_col]
    df['tempo_desde_ultimo_evento'] = np.nan

    for stay_id, grupo in df.groupby('stay_id'):
        estados = grupo['estado_atual'].values
        tempos = grupo['charttime'].values

        last_event_time = None
        tempo_desde = []

        for i, estado in enumerate(estados):
            if estado == 1:
                last_event_time = tempos[i]
                tempo_desde.append(0)
            else:
                if last_event_time is None:
                    tempo_desde.append(np.nan)
                else:
                    delta = (tempos[i] - last_event_time).astype('timedelta64[m]').astype(float)
                    tempo_desde.append(delta)

        df.loc[grupo.index, 'tempo_desde_ultimo_evento'] = tempo_desde

    df['duracao_evento'] = df.groupby('stay_id')['estado_atual'].transform(lambda x: x.expanding().mean())

    print("✅ Features de instabilidade criadas.")
    return df

In [27]:
def criar_features_intensidade(df, vars_continuas):
    df = df.sort_values(['stay_id', 'charttime']).reset_index(drop=True)

    novas_colunas = {}

    for var in vars_continuas:
        mask = ~df[var].isna()

        tempo_desde_ultima_medicao = np.full(len(df), np.nan)
        prop_medicoes = np.full(len(df), np.nan)

        for stay_id, grupo in df.groupby('stay_id'):
            tempos = grupo['charttime'].values
            mask_var = mask.loc[grupo.index].values

            last_meas_time = None
            tempos_desde = []
            contagem = 0

            for i, presente in enumerate(mask_var):
                if presente:
                    last_meas_time = tempos[i]
                    contagem += 1
                    tempos_desde.append(0)
                else:
                    if last_meas_time is None:
                        tempos_desde.append(np.nan)
                    else:
                        delta = (tempos[i] - last_meas_time).astype('timedelta64[m]').astype(float)
                        tempos_desde.append(delta)

            prop_medicoes_grupo = [contagem / (i + 1) for i in range(len(tempos))]

            tempo_desde_ultima_medicao[grupo.index] = tempos_desde
            prop_medicoes[grupo.index] = prop_medicoes_grupo

        novas_colunas[f'{var}_tempo_desde_ultima_medicao'] = tempo_desde_ultima_medicao
        novas_colunas[f'{var}_prop_medicoes'] = prop_medicoes

    df_novas = pd.DataFrame(novas_colunas, index=df.index)
    df = pd.concat([df, df_novas], axis=1)

    print("✅ Features de intensidade de medição criadas.")
    return df


In [ ]:
def extrair_features_cumulativas(df, exclude_cols=None, id_col='stay_id', time_col='charttime'):
    if exclude_cols is None:
        exclude_cols = []

    cols = [c for c in df.columns if c not in exclude_cols + [id_col, time_col] and not c.endswith('_mask')]
    df = df.sort_values([id_col, time_col]).reset_index(drop=True)
    df_feat = df.copy()

    for col in cols:
        df_feat[f'min_{col}'] = df_feat.groupby(id_col)[col].cummin().ffill()
        df_feat[f'max_{col}'] = df_feat.groupby(id_col)[col].cummax().ffill()

        n_meas = df_feat.groupby(id_col)[col].apply(lambda x: x.notna().cumsum()).reset_index(level=0, drop=True)
        cumsum = df_feat.groupby(id_col)[col].cumsum().ffill()

        df_feat[f'n_meas_{col}'] = n_meas
        df_feat[f'mean_{col}'] = cumsum / n_meas.replace(0, pd.NA)
        df_feat[f'mean_{col}'] = df_feat[f'mean_{col}'].fillna(method='ffill')

    print("✅ Features cumulativas extraídas.")
    return df_feat




In [ ]:
df_completo = processar_vasopressores_e_marcar_falencia(df_completo)

vars_validas = [var for var in vars_continuas if df_completo[var].isna().mean() < 0.5]
masks_validas = [v + '_mask' for v in vars_validas]

colunas_finais = vars_validas + masks_validas + ['stay_id', 'charttime', 'falencia']
df_final = df_completo[colunas_finais].copy()

print(f"✅ Dataset final pronto com {len(vars_validas)} variáveis contínuas válidas.")

In [29]:
def construir_janelas_temporais(df, jan_obs=36, jan_pred=12, passo=1, max_nan_ratio=0.5):
    candidate_cols = [
        col for col in df.columns
        if col not in ['stay_id', 'charttime', 'falencia'] and not col.endswith('_mask')
    ]
    vars_features = [col for col in candidate_cols if pd.api.types.is_numeric_dtype(df[col])]

    if len(vars_features) == 0:
        raise ValueError("Nenhuma variável numérica válida para construir janelas.")

    X, y, stays, times = [], [], [], []

    total_janelas = 0
    rejeitadas_nan = 0

    for stay_id, group in df.groupby('stay_id'):
        group = group.reset_index(drop=True)
        count_validas = 0

        max_start = len(group) - (jan_obs + jan_pred) + 1
        for i in range(0, max_start, passo):
            janela_obs = group.iloc[i:i + jan_obs]
            janela_pred = group.iloc[i + jan_obs:i + jan_obs + jan_pred]

            nan_ratio = janela_obs[vars_features].isna().mean().mean()
            total_janelas += 1

            if nan_ratio > max_nan_ratio:
                rejeitadas_nan += 1
                continue

            X.append(janela_obs[vars_features].values)
            y.append(int(janela_pred['falencia'].any()))
            stays.append(stay_id)
            times.append(janela_obs['charttime'].iloc[0])
            count_validas += 1

    print(f"\nTotal janelas avaliadas: {total_janelas}")
    print(f"Janelas rejeitadas por NaN: {rejeitadas_nan} ({rejeitadas_nan / total_janelas:.2%})")

    return np.array(X), np.array(y), stays, times


X, y, stays, times = construir_janelas_temporais(df_completo, max_nan_ratio=0.5)
print(f"✅ Janelas temporais criadas. Total: {len(X)}, X shape: {X.shape}, y shape: {y.shape}")

NameError: name 'df_completo' is not defined

In [ ]:
X_shape = X.shape
X_flat = X.reshape(-1, X_shape[2])

X_flat = np.nan_to_num(X_flat, nan=0.0, posinf=0.0, neginf=0.0)
X_flat = X_flat.astype(np.float64)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_flat).reshape(X_shape)

X = X_scaled

joblib.dump(scaler, "scaler.pkl")
print("✅ Padronização z-score aplicada.")

np.save("X.npy", X)
np.save("y.npy", y)
pd.DataFrame({"stay_id": stays, "start_time": times}).to_csv("janelas_metadata.csv", index=False)
print("✅ Dados exportados.")